# Task 0

### Process pcap files into Pandas Dataframe

In [1]:
import pandas as pd
import pyshark
import json

In [2]:
dns = pyshark.FileCapture('dns.pcap', keep_packets=True)

In [3]:
http = pyshark.FileCapture('http.pcap', keep_packets=True)

In [4]:
http2 = pyshark.FileCapture('http2.pcap', keep_packets=True)

In [5]:
try:
    i = 0
    while True:
        dns[i]
        i += 1
except KeyError:
    pass

In [6]:
try:
    i = 0
    while True:
        http[i]
        i += 1
except KeyError:
    pass

In [7]:
# try:
#     i = 0
#     while True:
#         http2[i]
#         i += 1
# except KeyError:
#     pass

In [76]:
print("Layer HTTP" in http[3][3])

TypeError: argument of type 'Layer' is not iterable

##### dns

In [9]:
dns_data = []
for pkt in dns:
    d = {
        'sniff_time': pkt.sniff_time,
        'length': pkt.length,
        'highest_layer': pkt.highest_layer,
        'time_stamp': pkt.sniff_timestamp,
        'ip': pkt.ip
#         'port': pkt[2].port
    }
    dns_data.append(d)

In [10]:
df_dns = pd.DataFrame(dns_data, columns = ['sniff_time', 'length', 'highest_layer', 'time_stamp', 'ip'])
df_dns.to_csv("dns_df.csv", index=False)

##### http

In [68]:
http_data = []
for pkt in http:
    print(pkt.layers)
#     print(len(pkt.layers))
    for i in range(len(pkt.layers)):
        print('#')
        if pkt.layers[i] == '<HTTP Layer>':
            print('hello')
#             d = {
#             'sniff_time': pkt.sniff_time,
#             'length': pkt.length,
#             'highest_layer': pkt.highest_layer,
#             'time_stamp': pkt.sniff_timestamp,
#             'ip': pkt.ip
#         #         'port': pkt[2].port
#             }
# http_data.append(d)

[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>]
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <DATA Layer>, <HTTP Layer>, <DATA-TEXT-LINES Layer>]
#
#
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>]
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Lay

[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>]
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>]
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>]
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>]
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>, <PNG Layer>]
#
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>, <PNG Layer>]
#
#
#
#
#
[<ETH Layer>, <IP L

[<ETH Layer>, <IP Layer>, <TCP Layer>, <DATA Layer>, <HTTP Layer>, <DATA-TEXT-LINES Layer>]
#
#
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>]
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <DATA Layer>, <HTTP Layer>, <DATA-TEXT-LINES Layer>]
#
#
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>, <HTTP Layer>]
#
#
#
#
[<ETH Layer>, <IP Layer>, <TCP Layer>]
#

In [ ]:
df_http = pd.DataFrame(http_data, columns = ['sniff_time', 'length', 'highest_layer', 'time_stamp'])
df_http.to_csv("http_df.csv", index=False)

In [ ]:
print(df_http.head())
print(len(df_http))

##### http2

In [ ]:
http2_data = []
for pkt in http2:
    d = {
        'sniff_time': pkt.sniff_time,
        'length': pkt.length,
        'highest_layer': pkt.highest_layer,
        'time_stamp': pkt.sniff_timestamp,
#         'ip': pkt.ip,
#         'port': pkt[2].port
    }
    http2_data.append(d)

In [ ]:
df_http2 = pd.DataFrame(http2_data, columns = ['sniff_time', 'length', 'highest_layer', 'time_stamp'])
df_http2.to_csv("http2_df.csv", index=False)

In [ ]:
print(df_http2.head())
print(len(df_http2))

### Process Operating System Query logs into Pandas Dataframe

In [13]:
log_list = []
with open("osqueryd.results.log", "r") as log_data:
    for line in log_data:
        log_list.append(eval(line))
df_log = pd.DataFrame(log_list)

In [77]:
df_log.to_csv("oslogs.csv", index=False)